In [ ]:
import sys
sys.path.append('../..')

import datetime as dt
from getpass import getpass
from glob import glob
from time import sleep

import ipyplot
from IPython.display import JSON
import rasterio
from rasterio.plot import show
import requests

from harmony_py.harmony import SpecialSession
from harmony_py.harmony import Client, Collection, Request


def download_and_show_results(harmony_job):
    status_url = harmony_job['links'][0]['href']
    print('Waiting for the job to finish')

    with SpecialSession(user='kbeam', pwd=getpass()) as session:
        while harmony_job['status'] != 'successful':
            harmony_job = session.get(status_url).json()
            print(f"Job is {harmony_job['status']}: {harmony_job['progress']}%")
            sleep(3)
    
        print('Downloading results')
        for result in [link for link in harmony_job['links'] if link['type'] == 'image/tiff']:
            with open(result['title'], 'wb') as f:
                print(f"  {result['title']}")
                f.write(session.get(result['href']).content)
                show(rasterio.open(result['title']))
                
def show_results():
    for fp in glob('*.tif'):
        show(rasterio.open(fp))

In [ ]:
request = Request(
    collection=Collection(id='C1234088182-EEDTEST'),
    spatial={
        'll': (52, -165),
        'ur': (77, -140)
    })

job1 = Client().submit(request, user='kbeam', pwd=getpass())
JSON(job1)

In [ ]:
request = Request(
    collection=Collection(id='C1234088182-EEDTEST'),
    temporal={
        'start': dt.datetime(2020, 6, 1),
        'stop': dt.datetime(2020, 6, 30)
    })

harmony_job = Client().submit(request, user='kbeam', pwd=getpass())
JSON(harmony_job)

In [ ]:
download_and_show_results(harmony_job)

In [ ]:
request = Request(
    collection=Collection(id='C1234088182-EEDTEST'),
    spatial={
        'll': (52, -165),
        'ur': (77, -140)
    },
    temporal={
        'start': dt.datetime(2010, 1, 1),
        'stop': dt.datetime(2020, 12, 30)
    })

harmony_job = Client().submit(request, user='kbeam', pwd=getpass())
JSON(harmony_job)

In [ ]:
download_and_show_results(harmony_job)
download_and_show_results(harmony_job)